https://github.com/0liu/ISLR/blob/master/Chapter%205%20Resampling%20Methods.ipynb

### data

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import scipy as sp
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_val_score
from sklearn.utils import resample
from sklearn.preprocessing import scale, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# from IPython.display import display
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
mpl.style.use("ggplot")

In [0]:
auto = pd.read_csv("/content/drive/My Drive/ISLR/data/Auto.csv", usecols = list(range(1,10)))

In [74]:
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


### **Model**

> #### split: training + test

Validation approach

In [0]:
X = auto[["horsepower"]]
y = auto[["mpg"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 196, random_state = 47)

In [76]:
auto_slr = LinearRegression()
auto_slr.fit(X_train, y_train)
y_pred = auto_slr.predict(X_test)
print("SLR MSE = ", mean_squared_error(y_test, y_pred))

SLR MSE =  25.273723992970933


다항 회귀: 2차식

In [0]:
poly2 = PolynomialFeatures(degree = 2)
X2 = poly2.fit_transform(X)
X2_train, X2_test, y_train, y_test = train_test_split(X2, y, test_size = 196, random_state = 47)

In [78]:
auto_poly2 = LinearRegression()
auto_poly2.fit(X2_train, y_train)
y2_pred = auto_poly2.predict(X2_test)
print("Polynomial regression of degree 2: MSE = ", mean_squared_error(y_test, y2_pred))

Polynomial regression of degree 2: MSE =  18.869003119495403


다항 회귀: 3차식

In [0]:
poly3 = PolynomialFeatures(degree = 3)
X3 = poly3.fit_transform(X)
X3_train, X3_test, y_train, y_test = train_test_split(X3, y, test_size = 196, random_state = 47)

In [80]:
auto_poly3 = LinearRegression()
auto_poly3.fit(X3_train, y_train)
y3_pred = auto_poly3.predict(X3_test)
print("Polynomial regression of degree 3: MSE = ", mean_squared_error(y_test, y3_pred))

Polynomial regression of degree 3: MSE =  18.833366996031767


Leave-One-Out Cross-Validation

In [81]:
auto_poly = LinearRegression()
loocv = LeaveOneOut()

for poly_deg in range(1, 6):
  print(f"\nPolynomial regression of degree {poly_deg}")
  poly = PolynomialFeatures(degree = poly_deg)
  X_d = poly.fit_transform(X)
  score = cross_val_score(auto_poly, X_d, y, cv = loocv, scoring = "neg_mean_squared_error")
  loocv_mse = score.mean()*(-1)
  print(f" MSE = {loocv_mse}")


Polynomial regression of degree 1
 MSE = 24.231513517929226

Polynomial regression of degree 2
 MSE = 19.248213124489745

Polynomial regression of degree 3
 MSE = 19.33498406411498

Polynomial regression of degree 4
 MSE = 19.424430307079398

Polynomial regression of degree 5
 MSE = 19.033198669299846


k-Fold Cross-Validation

In [82]:
auto_poly = LinearRegression()
kfold = KFold(n_splits = 10, random_state = 47)

for poly_deg in range(1, 11):
  print(f"\nPolynomial regression of degree {poly_deg}")
  poly = PolynomialFeatures(degree = poly_deg)
  X_d = poly.fit_transform(X)
  score = cross_val_score(auto_poly, X_d, y, cv = kfold, scoring = "neg_mean_squared_error")
  kfold_mse = score.mean()*(-1)
  print(f" MSE = {kfold_mse}")


Polynomial regression of degree 1
 MSE = 27.439933652339864

Polynomial regression of degree 2
 MSE = 21.235840055801567

Polynomial regression of degree 3
 MSE = 21.336606183382038

Polynomial regression of degree 4
 MSE = 21.353886969306874

Polynomial regression of degree 5
 MSE = 20.905558736691837

Polynomial regression of degree 6
 MSE = 20.780544653507675

Polynomial regression of degree 7


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 MSE = 20.952970598113758

Polynomial regression of degree 8
 MSE = 21.077108146457544

Polynomial regression of degree 9
 MSE = 21.035590598116325

Polynomial regression of degree 10
 MSE = 20.978001582517084


> #### Bootstrap

In [0]:
portfolio = pd.read_csv("/content/drive/My Drive/ISLR/data/Portfolio.csv", usecols = list(range(1,3)))

In [84]:
print(portfolio.head(), "\nshape: ", portfolio.shape)

          X         Y
0 -0.895251 -0.234924
1 -1.562454 -0.885176
2 -0.417090  0.271888
3  1.044356 -0.734198
4 -0.315568  0.841983 
shape:  (100, 2)


In [0]:
# X, Y라는 이름을 가진 데이터프레임일 것.
def alpha(data):
  sigma = data.cov()
  var_x = sigma.X["X"]
  var_y = sigma.Y["Y"]
  cov_xy = sigma.X["Y"]
  return (var_y - cov_xy) / (var_x + var_y - 2*cov_xy)

In [86]:
print("Portfolio alpha = ", alpha(portfolio))

Portfolio alpha =  0.57583207459283


alpha 예측값을 봤으니 본격적인 bootstrap을 이용해 alpha 모수 추정 (1000번)

In [0]:
N = portfolio.shape[0]; B = 1000
portfolio_b = resample(portfolio, n_samples = N*B, random_state = 42)

In [0]:
alphas = [alpha(group) for name, group in portfolio_b.groupby(np.arange(N*B) // N)]
# alphas = []
# for name, group in portfolio_b.groupby(np.arange(N*B) // N):
#   alphas.append(alpha(group))

In [89]:
alpha_bias = np.mean(alphas) - alpha(portfolio)
alpha_se = np.std(alphas)
alpha_bootstrap = pd.DataFrame([[alpha(portfolio), alpha_bias, alpha_se],], index = ["alpha"])
alpha_bootstrap.columns = ["original", "bias", "std error"]
alpha_bootstrap

,original,bias,std error
alpha,0.575832,0.001963,0.089929


단순 회귀에서 coefficient

In [0]:
# features는 칼럼 불러오기 때문에 리스트일 것, response는 보통 하나, 이러면 당연하게도 data는 데이터프레임.
def auto_coef(data, features, response):
  reg = LinearRegression()
  reg.fit(data[features], data[response])
  return [reg.intercept_] + list(reg.coef_)

In [91]:
coef_original = pd.DataFrame([auto_coef(auto, ["horsepower"], "mpg")], columns = ["Intercept", "horsepower"], index = [""])
print("mpg ~ horsepower coefficients\n", coef_original)

mpg ~ horsepower coefficients
   Intercept  horsepower
  39.935861   -0.157845


이번에도 당연히 bootstrap으로 회귀계수에 대해 알아보자 (1000번)

In [0]:
N = auto.shape[0];  B = 1000
auto_b = resample(auto, n_samples = N*B, random_state = 42)

In [0]:
coefs = [auto_coef(group, ["horsepower"], "mpg") for name, group in auto_b.groupby(np.arange(N*B) // N)]
# coefs = []
# for name, group in auto_b.groupby(np.arange(N*B) // N):
#   coefs.append(auto_coef(group, ["horsepower"], "mpg"))
coefs = pd.DataFrame(coefs, columns = ["Intercept", "horsepower"])

In [94]:
coef_bias = coefs.mean() - coef_original
coef_se = coefs.std()
coef_bootstrap = pd.concat([coef_original.T.copy(), coef_bias.T, coef_se], axis = 1)
coef_bootstrap.columns = ["original", "bias", "std error"]
coef_bootstrap

,original,bias,std error
Intercept,39.935861,0.033521,0.869087
horsepower,-0.157845,-0.000500,0.007503
